In [ ]:
%matplotlib inline
import pandas as pd
import nupis
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
import matplotlib as mpl
import geopandas as gp
import pysal as ps
import pysal.contrib.viz.mapping as maps
import palettable

pd.options.mode.chained_assignment = None

sns.set(style='whitegrid', palette='bright', context='notebook')

### Setores censitários

Para baixar as malhas dos outros estados do Brasil ir em:

ftp://geoftp.ibge.gov.br/organizacao_do_territorio/malhas_territoriais/malhas_de_setores_censitarios__divisoes_intramunicipais/censo_2010/setores_censitarios_shp/

Para baixar o arquivo com as variáveis por setores censitários em MG:

ftp://ftp.ibge.gov.br/Censos/Censo_Demografico_2010/Resultados_do_Universo/Agregados_por_Setores_Censitarios/

Dicas em: http://www.analisededadosempython.org/espacial

In [ ]:
# estado de Minas Gerais dividido em setores censitários
setores = gp.read_file('31SEE250GC_SIR.shp') 
setores.head(3)

In [ ]:
setores.shape

In [ ]:
# a partir do shapefile de setores de MG, selecionar o município de Varginha e apenas os setores urbanos
vga = setores.query('NM_MUNICIP == "VARGINHA"')
mapa_vga = vga.query('TIPO == "URBANO"')
mapa_vga.shape

In [ ]:
mapa_vga.columns

In [ ]:
# plotar os bairros, porém, a divisão está em setores censitários - as 161 linhas
# (há bairros repetidos)
mapa = mapa_vga
ax = mapa.plot(figsize=(15,15))

for i in mapa.index:
    plt.text(mapa.geometry.centroid[i].coords[0][0], mapa.geometry.centroid[i].coords[0][1], mapa.NM_BAIRRO[i],
             fontsize=10, horizontalalignment='center', verticalalignment='center')

ax.set_axis_off();

In [ ]:
# tentar agrupar em bairros
# mapa contém as informações divididas em 161 setores censitários e queremos unir os setores em 38 bairros
# uso da função dissolve 
# http://geopandas.org/aggregation_with_dissolve.html
bairros = mapa_vga.dissolve(by='NM_BAIRRO')
bairros.head()

In [ ]:
# plotar os bairros usando o novo dataframe bairros
# Obs.: os índices ficaram como os nomes dos bairros, por isso, criei uma nova coluna NM_BAIRRO para
# colocar os nomes dos bairros e a nova coluna index irá de 0 a 37 (número de bairros)
bairros['NM_BAIRRO'] = bairros.index
bairros.index = range(len(bairros))
bairros.head()

In [ ]:
# plotar os bairros
ax = bairros.plot(figsize=(15,15), cmap='Accent')

for i in bairros.index:
    plt.text(bairros.geometry.centroid[i].coords[0][0], bairros.geometry.centroid[i].coords[0][1], bairros.NM_BAIRRO[i],
             fontsize=10, horizontalalignment='center', verticalalignment='center')

ax.set_axis_off();

## Plotar os nomes dos bairros de outro município de MG:

In [ ]:
# para saber quais municípios possuem os nomes dos bairros disponíveis
setores.NM_BAIRRO.fillna('nnnn', inplace=True)  # preencher os que não têm nomes dos bairros com a string 'nnnn'
setores = setores.query('NM_BAIRRO != "nnnn"')

In [ ]:
# retornar esses municípios
setores.query('NM_BAIRRO != "nada"')['NM_MUNICIP'].unique()

## Plotar os nomes dos bairros de outro município de MG:

In [ ]:
# plotar o mapa sem os nomes



In [ ]:
# plotar o mapa com os nomes


### Agora vamos pegar variáveis sobre os setores censitários de MG e utilizar os de Varginha.

O arquivo que usaremos será o 'basico_MG.csv'

Obs.: peguei esse arquivo em: 

ftp://ftp.ibge.gov.br/Censos/Censo_Demografico_2010/Resultados_do_Universo/Agregados_por_Setores_Censitarios/).

As variáveis desse arquivo estão codificadas, para saber o que significam, consultar o arquivo 'base_setores_censitarios_censo_2010.pdf'

In [ ]:
# carregar arquivo com dados sobre os setores censitários de MG
mg = pd.read_csv('basico_MG.csv', encoding='latin1', sep=';', decimal=',')
mg.head()

In [ ]:
mg.columns

In [ ]:
mg.Nome_do_municipio.unique()

In [ ]:
# informações dos setores censitários de Varginha
mg.query('Nome_do_municipio == "VARGINHA"')

In [ ]:
# salvar as informações de Varginha num dataframe
vga_df = mg.query('Nome_do_municipio == "VARGINHA"')
vga_df = vga_df.query('Situacao_setor == [1, 2, 3]')   # só setores urbanos
vga_df.head()

In [ ]:
vga_df.shape    # 161 setores censitários

In [ ]:
vga_df.columns   # variáveis: 12

In [ ]:
# o objeto 'mapa_vga' contém as informações geográficas de todos os setores censitários de Varginha
# vamos incluir as informações das variáveis presentes em 'vgn_df' neste dataframe
# e chamá-lo de geodf 
# como usaremos a coluna de código do setor censitário como a variável para unir os dois conjuntos de
# dados, o nome dessa coluna deve ser o mesmo nos dois (chamarei de 'Cod_setor')
df = vga_df
geodf = mapa_vga
geodf.geometry = geodf.geometry.simplify(0.001)
geodf.rename(columns={'CD_GEOCODI': 'CD_setor'}, inplace=True)
geodf.head()

In [ ]:
geodf.shape

In [ ]:
# os códigos não estão sendo lidos como inteiros, transformá-los em inteiros
geodf.CD_setor = geodf.CD_setor.astype('int64')

In [ ]:
# para que os dois dataframes tenham a coluna de códigos com o mesmo nome,
# alterar para CD_setor
df.rename(columns={'Cod_setor': 'CD_setor'}, inplace=True)

In [ ]:
# unir o dataframe de informações geográficas com o outro dataframe com as variáveis
# Obs.: 'suffixes' é para o caso de haver variáveis de mesmo nome, incluir alguma coisa no final do nome
# em cada dataframe
varginha = pd.merge(geodf, df, on='CD_setor')
varginha

In [ ]:
varginha.columns